<a href="https://colab.research.google.com/github/poziryna84/Digit_Recognizer_Kaggle/blob/master/Webscrapping_symptoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from urllib.request import urlopen
import pandas as pd 
from bs4 import BeautifulSoup
import urllib
import ssl
import time

In [0]:
ssl._create_default_https_context = ssl._create_unverified_context

In [0]:
url = "https://www.doctorslounge.com/forums/"

In [0]:
html = urlopen(url)

In [0]:
res = BeautifulSoup(html.read(),"html5lib")

In [0]:
tags = res.findAll('a', href=True)

In [0]:
row_strings = []
generals_urls = []
general_topics = []
for i in tags:
  if (str(i)).startswith('<a href="./viewforum.php?'):
    row_strings.append(str(i))
    generals_urls.append(url + (str(i)).replace('<a href="./', '').split('">')[0])
    general_topics.append((str(i)).replace('</a>', '').split('>')[-1])

In [9]:
print(row_strings[7])
print(generals_urls[7])
print(general_topics[7])

<a href="./viewforum.php?f=106&amp;sid=c32a1bf0b73ffd5861a5197a986b7fbf">Cardiology</a>
https://www.doctorslounge.com/forums/viewforum.php?f=106&amp;sid=c32a1bf0b73ffd5861a5197a986b7fbf
Cardiology


In [10]:
df_general = pd.DataFrame([row_strings, generals_urls, general_topics]).T
df_general.columns = ['row_strings', 'generals_urls', 'general_topics']
df_general.head(5)

,row_strings,generals_urls,general_topics
0,"<a href=""./viewforum.php?f=124&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Surgery
1,"<a href=""./viewforum.php?f=110&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Kidney disorders (Nephrology)
2,"<a href=""./viewforum.php?f=126&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Blood disorders (Hematology)
3,"<a href=""./viewforum.php?f=123&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Hormone disorders (Endocrinology)
4,"<a href=""./viewforum.php?f=109&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Pharmacy &amp; Drugs


In [0]:
df_general = df_general[(df_general['general_topics'] !='Pharmacy &amp; Drugs') & 
                        (df_general['general_topics'] != 'Job Center') &
                        (df_general['general_topics'] != 'Miscellaneous Medical Topics') &
                        (df_general['general_topics'] != 'Medical Students') &
                        (df_general['general_topics'] != 'Pediatrics')]

In [0]:
filtered_general_urls = list(df_general['generals_urls'])

In [0]:
lista_url = []
generals_urls1 = []

for i in filtered_general_urls:
  url_gen = i
  html_gen = urlopen(url_gen)
  res_gen = BeautifulSoup(html_gen.read(),"html5lib")
  tags_gen = res_gen.findAll('a', href=True)
  for e in tags_gen:
    if (str(e)).startswith('<a class="forumtitle" href="./viewforum.php') or (str(e)).startswith('<a class="topictitle" href="./viewtopic.php'):
      string = str(e)
      lista_url.append(string)
      generals_urls1.append(i)

 

In [0]:
lista_url_ = [url + i.replace('<a class="forumtitle" href="./', '').replace('<a class="topictitle" href="./', '').split('">')[0] for i in lista_url]
lista_nombres = [e.replace("</a>", "").split(">")[-1] for e in lista_url]


In [15]:
print(lista_url_[7])
print(lista_nombres[7])
print(generals_urls1[7])

https://www.doctorslounge.com/forums/viewforum.php?f=87&amp;sid=5542beed1328135aa572b66cf760f2cd
Hematology Topics
https://www.doctorslounge.com/forums/viewforum.php?f=126&amp;sid=c32a1bf0b73ffd5861a5197a986b7fbf


In [16]:
df_specific = pd.DataFrame([lista_nombres, lista_url_, generals_urls1]).T
df_specific.columns = ['specific_topics', 'specific_url', 'generals_urls_aux']
df_specific.head(5)

,specific_topics,specific_url,generals_urls_aux
0,Surgery Topics,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...
1,Renal Failure,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...
2,Nephrotic Syndrome,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...
3,Glomerulonephritis,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...
4,Dialysis,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...


In [0]:
df_extended = pd.merge(df_specific, df_general, how = 'left',
                       left_on = 'generals_urls_aux',
                       right_on = 'generals_urls')


In [0]:
topics_in_specific = []
specific_url = []
for i in lista_url_:
  url_gen = i
  html_gen = urlopen(url_gen)
  res_gen = BeautifulSoup(html_gen.read(),"html5lib")
  tags_gen = res_gen.findAll('a', href=True)
  for e in tags_gen:
    string = str(e)
    if string.startswith('<a class="topictitle" href="./viewtopic.php'):
      topics_in_specific.append(string)
      specific_url.append(i)


In [0]:
topics_inspec_links = [url + i.replace('<a class="topictitle" href="./', '').split('&amp;sid')[0].replace('amp;', '') for i in topics_in_specific] 

In [20]:
df_last_url = pd.DataFrame([specific_url, topics_inspec_links]).T
df_last_url.columns = ['url_aux', 'texts_url']
df_last_url.head()

,url_aux,texts_url
0,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...
1,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...
2,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...
3,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...
4,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...


In [0]:
df_last_url = pd.merge(df_last_url, df_extended, how = 'left',
                       left_on = 'url_aux',
                       right_on = 'specific_url')

In [22]:
df_last_url.head(3)

,url_aux,texts_url,specific_topics,specific_url,generals_urls_aux,row_strings,generals_urls,general_topics
0,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...,Surgery Topics,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...,"<a href=""./viewforum.php?f=124&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Surgery
1,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...,Surgery Topics,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...,"<a href=""./viewforum.php?f=124&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Surgery
2,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewtopic...,Surgery Topics,https://www.doctorslounge.com/forums/viewforum...,https://www.doctorslounge.com/forums/viewforum...,"<a href=""./viewforum.php?f=124&amp;sid=c32a1bf...",https://www.doctorslounge.com/forums/viewforum...,Surgery


In [0]:
  html_gen = urlopen('https://www.doctorslounge.com/forums/viewtopic.php?f=84&t=50736')
  res_gen = BeautifulSoup(html_gen.read(),"html5lib")
  tags_gen = res_gen.findAll('div', {'class':'content'})

In [0]:
row_texts = []
url_aux = []
for i in list(df_last_url.texts_url):
#  t0 = time.time()
  html_gen = urlopen(i)
  res_gen = BeautifulSoup(html_gen.read(),"html5lib")
  tags_gen = res_gen.findAll('div', {'class':'content'})
  string = str(tags_gen)
  row_texts.append(string.split('<div class="content">')[1].replace('<br/>', '').replace('</div>','').replace('\n', '').replace(']', ''))
  url_aux.append(i)
#  response_delay = time.time() - t0
#  time.sleep(5*response_delay)

In [26]:
row_texts[5000]

'First off I don\'t know if this is the proper category for this question....sorry if it isn\'t.Ok, I\'m 26 years of age never touched alcohol until I was at the age of 23 , I\'m in no way an alcoholic , I drink only on the weekend and now down to only one day a week. I go out with my friends we have drinks we all usually get "drunk"...now here\'s my question...When I first started drinking it effected me almost not at all, if always be clear headed when drinking .... I could drink all night if I wanted to and it would almost never effect me ...unless I drank stupidly and just blacked out when I\'ve never done... Well about a year ago I was drinking as usual the night was fine I came home when to bed and it seemed like something drastically switched inside of me .... Not like a hangover feeling like something else was wrong... From that day on , when I drink it effects me a lot... I get drunk much quicker ... I always feel bad the next day when before I felt no hangover effects ....and

In [0]:
df_texts = pd.DataFrame([row_texts, url_aux]).T
df_texts.columns = ['row_text', 'row_text_aux']

In [30]:
df_texts.head(5)

,row_text,row_text_aux
0,I had my leg fractured in 2005; fibula was bro...,https://www.doctorslounge.com/forums/viewtopic...
1,"Hello. I am a young woman with cerebral palsy,...",https://www.doctorslounge.com/forums/viewtopic...
2,"Hello, I’m a 59-year-old Caucasian female in o...",https://www.doctorslounge.com/forums/viewtopic...
3,I have had two hernia surgeries years ago. The...,https://www.doctorslounge.com/forums/viewtopic...
4,I'm not sure if this is normal or not. It feel...,https://www.doctorslounge.com/forums/viewtopic...


In [0]:
final_df = pd.merge(df_last_url, df_texts, how = 'left',
                    left_on = 'texts_url',
                    right_on = 'row_text_aux')


In [0]:
df_row = final_df.to_excel('symptoms_row.xlsx')